In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import math

### Workbook: Identify Bubbles in FTSE AIM Index

Layout of this workbook into the following sections

01. Section to import tickers for the FTSE AIM Index
   
02. Choosing which trends to identify: positive bubbles, negative bubbles, etc.

03. Running the analysis

04. Presenting Results of analysis

05. Downloading Results in correct format for a report

**How the model works**

- The model works by only looking at historical price data of a security
- The model takes historical price data of a stock, over a fixed period
- Then tries to fit a curve to the data over varying time windows
- The curves look for future bubbles in the stock price (for example positive bubbles where price increases rapidly)
- Time windows are calculated by dividing the historical data into segments (i.e 8 individual weeks of a 2 month dataset)
- The confidence in the future projected trend is determined by analysing successive segments of time series data and seeing how many of them give the same extrapolated fit.


**Purpose of the report**

I have two ideas in mind for the report, a report on publically traded insurance company stocks or a report on the small/micro cap FTSE AIM Index. Both sectors presumably have little analysis produced on them, and a report may add value and or be able to identify trends in prices.

- A report can be produced for either group
- The report will highlight if certain companies show evidence that there is a bubble forming in the stock price
- The report should be easy to run on a monthly basis
- By making forward looking predictions the validity of this analysis can be assessed.
- It may also be possible to look at if this analysis can generate a return (by incorporating options price data)
- Report can then be shared with others

## Section 01.1 : Importing Tickers

In [ ]:
'''
Screen FTSE Aim Index Companies for bubbles, tickers may need ".L" added onto them

The a priori method of determining bubbles should still apply

Yfinance module may have issues when trying to receive data for this many companies.

Investigate whether there are other libraries which can do bulk downloads without having your IP Address blocked
'''

ftse_aim_df = pd.read_csv('FTSE_AIM_TICKERS.csv', encoding = "iso-8859-1")
tickers = ftse_aim_df['Ticker'].tolist()
stocks = ftse_aim_df['Name'].tolist()
tickers_l = [t + ".L" for t in tickers]

df = np.array([stocks,tickers_l])
print("The array of FTSE AIM stocks with their tickers is %s" % df)

In [ ]:
'''
For a trial run of the workbook, just two tickers will be used:
'''
insurance_tickers = pd.read_csv('Insurance_Companies.csv', encoding = "utf-8")

insurance_names = insurance_tickers['Name'].to_list()
insurance_stocks = insurance_tickers['Stock'].to_list()

print(insurance_names)
print(insurance_stocks)

## Section 01.2 : Importing Libraries and Data Download

In [ ]:
import lppls_funcs_fix as lppls_funcs # Use my updated python file, with new calculation of LPPLS Confidence Indicators
import ipywidgets as widgets
from IPython.display import display, clear_output
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

**Now select the length of time to conduct the bubble analysis over**

- Historical stock prices can be downloaded over a varying time period
- Using a longer time period will slow down the run time of the model
- Granularity of the dataset can be varied to download prices quoted: weekly, daily, hourly, etc.
- For practicality reasons an hourly dataset of prices over a 1 to 3 month period should be a good compromise, which balances usefulness of the analysis and speed of run time.

In [ ]:
duration_widget = widgets.RadioButtons(
    options=['1mo', '2mo', '3mo','6mo','1y','2y'],
    value = '1y', #defaults to 1 year
    description='Months of historical data to use in analysis:',
    disabled=False
)

display(duration_widget)

In [ ]:
# Full Range of granularities: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo

granularity_widget = widgets.RadioButtons(
    options=['1wk', '1d' ,'1h'], # hourly prices are likely unavailable for insurance stocks as they're not very liquid
    value = '1d', #defaults to 1 point per day
    description='Granularity of historical data, prices quoted:',
    disabled=False
)

display(granularity_widget)


**Choose Stock to analse within the workbook**

In [ ]:
stock = "BEZ.L"

In [ ]:
# Download the data for a single stock
# To download for multiple stocks enter "SPY QQQ VWRP"

data = yf.download(insurance_stocks, period = duration_widget.value, interval = granularity_widget.value)
time = [pd.Timestamp.toordinal(t1) for t1 in data.index]
prices = [i[0] for i in data['Close'].values.tolist()]

## Section 01.3 : Saving YFinance Download to CSV & Overwriting current CSV

In [ ]:
pd_closes = pd.DataFrame(data['Close'])
pd_closes_long = pd_closes.reset_index().melt(id_vars=['Date'], var_name='Ticker', value_name='Prices')
pd_closes_long['Date'] = pd_closes_long['Date'].astype(str)

In [ ]:
current_long_data = pd.read_csv('Bubble_Data/price_data.csv')
current_long_data

In [ ]:
combined_data = pd.concat([pd_closes_long, current_long_data])

combined_data = combined_data.drop_duplicates(subset=['Date', 'Ticker']).reset_index(drop=True)

combined_data

**Add new data to CSV**

In [ ]:
pd_closes_long.to_csv('Bubble_Data/price_data.csv', index = False)
combined_data.query("Ticker == 'AIG'")

In [ ]:
for i in range(0, len(insurance_stocks)):
    globals()[f'var_{i:02}'] = None

# Example: Assigning values to the dynamically created variables
for i in range(0, len(insurance_stocks)):
    t = combined_data[combined_data['Ticker'] == insurance_stocks[i]]['Date']
    time = [pd.Timestamp(t1).toordinal() for t1 in t.values]
    p = combined_data[combined_data['Ticker'] == insurance_stocks[i]]['Prices']
    globals()[f'var_{i:02}'] = lppls_funcs.LPPLS(observations = np.array([time,p]))

# Printing the values of the dynamically created variables
for i in range(0, len(insurance_stocks)):
    print(f'var_{i:02} =', globals()[f'var_{i:02}'])

## Section 02.1: Running the Model


**Select how many stocks to run the model for**

The initial run of the model will produce a best fit of the log period power law singularity curve for the time series data.

At this stage you need to select how many individual stocks to run the next set of analysis for:
- Select from the drop down below the number of stocks to compute the analysis for.
- The plots will show the graphical outputs for a single stock for reference.
- The time intensive part of the model is the calculation of the future bubble likelihoods.

In [ ]:
MAX_SEARCHES = 25
observations = np.array([time, prices])

# Creates a new LPPLS model for a specific set of price and time series data
lppls_model = lppls_funcs.LPPLS(observations=observations)
#indicators_medium = lppls_model.compute_indicators(res_medium)

# Fit the model to the data and get back the parameters
tc, m, w, a, b, c, c1, c2, O, D = lppls_model.fit(MAX_SEARCHES)

# Visualize the fit
lppls_model.plot_fit()

print("The critical time is: %s" % pd.Timestamp.fromordinal(int(round(tc, 0))))

In [ ]:
example_fit = indicators_medium._fits[229][9]

lppls_test = lppls_funcs.LPPLS(observations = observations)

test_fit = lppls_test.lppls(
    observations[0],
    example_fit['tc'],
    example_fit['m'],
    example_fit['w'],
    example_fit['a'],
    example_fit['b'],
    example_fit['c1'],
    example_fit['c2']
)

print(example_fit['tc'])
print("The example fit's critical time is: %s" % pd.Timestamp.fromordinal(int(round(example_fit['tc'], 0))))
print(example_fit['t2'])
print("The example fit's window end time is: %s" % pd.Timestamp.fromordinal(int(round(example_fit['t2'], 0))))

In [ ]:
from matplotlib import pyplot as plt

t_obs = [pd.Timestamp.fromordinal(d) for d in observations[0, :].astype('int32')]
fig, (ax1) = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(14, 8))

ax1.plot(t_obs, observations[1], label='price', color='black', linewidth=0.75)
ax1.plot(t_obs, test_fit, label='lppls fit', color='blue', alpha=0.5)
ax1.grid(which='major', axis='both', linestyle='-.')
ax1.set_ylabel('ln(price)')
ax1.set_xlabel('Date')
plt.axvline(x=pd.Timestamp.fromordinal(int(round(example_fit['t2'], 0))), label='t2={}'.format(pd.Timestamp.fromordinal(int(round(example_fit['t2'],0)))), linestyle = '--', color = 'red')
plt.axvline(x=pd.Timestamp.fromordinal(int(round(example_fit['tc'], 0))), label='tc={}'.format(pd.Timestamp.fromordinal(int(round(example_fit['tc'],0)))), linestyle = '--', color = 'green')
plt.xticks(rotation=10)
ax1.legend(loc=2)

## Section 02.2 Computing Confidence Indicators

**Now compute the likelihood of a future bubble forming**

- Select from the drop down whether you want to identify short or medium term bubbles in the stock prices.
- Short term bubbles consider 1 to 6 months of historical data.
- Medium term bubbles consider up to 18 months of historical data.
- Identifying a long term bubble can be too time intensive.

In [ ]:
'''
res_medium = lppls_model.mp_compute_nested_fits(
    workers=16,
    window_size=90,
    smallest_window_size=10, 
    outer_increment=1,
    inner_increment=1, 
    max_searches=25
)
'''
# In order to save run time res_medium is calculated once
# Then saved using pickle library
# Then imported again from pickle, as computing fits can take a long time

indicators_medium

lppls_model.plot_confidence_indicators(res_medium)

## Section 02.3 Getting critical times of the bubbles

**The bubble critical time is the time at which the bubble peaks**

- This is the likely time for when the price of the stock reaches its maximum/minimum for positive/negative bubbles
- In this section we show a table of results for each stock, with the critical time, bubble size and scenario probability of the future development of the bubble.
- You can also select the analysis below to show the extrapolated plot of the future stock price curve of the predicted bubble.

**K Means Clustering**

The k-means clustering calculation that is performed in the (t<sub>2</sub> - t<sub>1</sub>, t<sub>c</sub>-t<sub>2</sub>) space.
The analysis is performed for a single value of t<sub>2</sub>, i.e on a single collection of fits extrapolated from the same day.
This k-means clustering analysis is conducted on all the qualifying fits from that day.

Each cluster will represent a different possible future scenario that is the best fit of the collection of predictions
- For each cluster, calculate average of critical time (mu<sub>tc</sub>) and the standard deviation of time (sigma<sub>tc</sub>)
- We calculate the scenario probability as the fraction of fits that fall into the cluster
- Only the qualified fits get put into clusters

https://ethz.ch/content/dam/ethz/special-interest/mtec/chair-of-entrepreneurial-risks-dam/documents/FCO/appendix-FCO-ETH-SIMAG.pdf



We would expect that we have more confidence in a bubble if it can be detected across all windows

In our standard example we calculate the probabilities of bubbles at dates, considering data from a window of 120 to 30 days prior.
We use an inner increment of 3 days, meaning the 90 day duration of the window is divided into 30 sub-windows
Bubble pos_conf from the model is calculated as the share of positive bubbles (where b>0) which are qualified (have valid values for all parameters), divided by the number of positive bubbles in general.
Why is the the share of positive bubbles not divided by the share of all 

In [ ]:
# Array with date, critical time, bubble likelihood

# 1. Get positive bubble dates with at least 3 qualifying fits (so that k means clustering can be ran)
qualifying_count = []
for date in indicators_medium._fits:
    qual_count = 0
    for fit in date:
        if fit['is_qualified'] == True:
            qual_count += 1
    qualifying_count.append(qual_count)

fits = [count > 2 for count in qualifying_count]
high_conf_df = indicators_medium[fits]

# 2. append to matrix, a col with just the tc values



In [ ]:
def positive_bubble_times(df):
    '''
    Function takes a dataframe with all the confidence indicators for a dataset, 
    then will add an additional column with time-space coordinates for the qualifying positive
    bubble fits.
    
    Inputs:
    df (pandas dataframe), the compute_indicators() output
    
    Outputs:
    df_out (pandas dataframe), the compute_indicators() output, with additional columns of 'time array'
        and 'tc'.
    
        'time array' (array), column of arrays of all the coordinates of the qualifying
            fits for the positive bubbles detected.
    
            Coordinates are in the (t2 - t1, tc - t2) space.
                
        'tc' (float), column with float values for the critical time, the time when the bubble peaks.
    '''
    fits = df._fits
    vars = []
    for dicts in fits:
        t_list = []
        total_fits = len(dicts)
        for element in dicts:
            if element['is_qualified'] == True and element['b'] < 0 :
                t_list.append((element['t2'] - element['t1'], element['tc'] - element['t2']))
        total_qualifying = len(t_list)
        vars.append((total_qualifying ,t_list))
    
    compute_df = [*zip(*vars)]
    df_out = df.copy()
    df_out.loc[:,'qualifying_num'] = compute_df[0]
    df_out.loc[:,'time_array'] = compute_df[1]
    return(df_out)

pos_array_df = positive_bubble_times(high_conf_df)

In [ ]:
pos_array_df

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

def get_clusters(max_clusters, df):
    '''
    Function analyses the predicted bubble scenarios produced for each qualifying fit of the data,
    all qualifying fits of the data have a critical time (the time at which the bubble peaks), as
    well as bubble start time t2, and the start time of the historical data period used to fit the
    curve t1.

    Qualifying fits when plotted in the (t2- t1, tc-t2) space will appear as points near eachother
    if the predicted bubbles peak at the same time in the future.

    Hence we use k means clustering to figure out how many different clusters (groups of fits which
    represent the same future bubble

    Inputs:
    df (pandas dataframe), the time_arrays values from the compute_indicators() function output. 

    max_clusters (integer), the number of clusters to identify fits

    Outputs:
    df_output (dataframe), a dataframe with the same number of rows as df. It has three columns:
    
        clusters (integer), the number of clusters detected, for which the qualifying fits predict similar bubbles.

        cluster_coords (array), an array of tuples, the coordinates in the (t2 - t1, tc - t2) space that are
            located within the largest cluster. If multiple clusters have the same number of points within
            them, then array is made up of multiple sub-arrays, each a group of coordinates.
    '''
    range_clusters = range(2, max_clusters)
    cluster_num = []
    coords = []
    for array in df:
        ###
        # NOTE : Include check here that number of qualifying fits in 'array' is >= max_clusters
        ###
        
        if len(array) < max_clusters:
            raise ValueError("One time array only has %s qualifying fits, the minimum is %s" % (len(array), max_clusters))
        ###
        silhouette_avg = []
        for n in range_clusters:
            kmeans = KMeans(n_clusters=n, random_state=0, n_init="auto").fit(array)
            silhouette_avg.append(silhouette_score(array, kmeans.labels_))

        silhouette_index = silhouette_avg.index(max(silhouette_avg))
        silhouette_optimum_n = silhouette_index + 2 # as minimum number of clusters in k means analysis is 2
        cluster_num.append(silhouette_optimum_n)
        
        # With number of clusters, we now get the coordinates within the largest cluster
        kmeans_opt = KMeans(n_clusters = silhouette_optimum_n, random_state = 0, n_init = "auto").fit(array)
        #1. calculate largest cluster
        cluster_ints = kmeans_opt.labels_.tolist()
        all_modes = [i for i in set(cluster_ints) if cluster_ints.count(i) == max(map(cluster_ints.count,cluster_ints))]

        #2. get index of largest cluster, from kmeans.labels_
        #3. get coords if (cluster index == largest cluster index)
        multiple_modes = []
        for j in all_modes:
            coords_i = [array[i] for i in range(0,len(array)) if kmeans_opt.labels_[i] == j]
            multiple_modes.append(coords_i)
        coords.append(multiple_modes)

    df_output = pd.DataFrame(
        {'clusters' : cluster_num,
         'cluster_coords' : coords
        },
        index = df.index.values.tolist()
    )
    return(df_output)

In [ ]:
cluster_df = get_clusters(3, pos_array_df.time_array)
combined_df = pd.concat([pos_array_df, cluster_df], axis = 1)

combined_df

In [ ]:
# Calculate mu tc and sigma tc

def calculate_averages(cluster_dfs):
    '''
    Function calculates the time into the future the critical time of the bubble is expected to occur at, as well
    as the error on the estimate of the critical time.

    The bubble is expected to peak at tc, the average of all forecasts that are qualifying fits, is taken as 
    mu_tc_minus_t2.

    The error on this estimate of the critical time minus current time, is taken as sigma, the standard deviation
    of all the qualifying fits for each bubble cluster.

    Inputs:

    Outputs:
    
    '''
    
    cluster_coords = cluster_dfs['cluster_coords']
    mu_vals = []
    sigma_vals = []
    for row in cluster_coords:
        if(len(row) == 1):
            mu_tc_minus_t2 = sum(v[1] for v in row[0]) / len(row[0])
            mu_squared = sum(v[1]**2 for v in row[0]) / len(row[0])
            variance = mu_squared - mu_tc_minus_t2**2
            sigma = math.sqrt(variance)
            mu_vals.append(mu_tc_minus_t2)
            sigma_vals.append(sigma)
        else:
            averages = []
            sigmas = []
            for cluster in row:
                mu_tc_minus_t2 = sum(v[1] for v in cluster) / len(cluster)
                mu_squared = sum(v[1]**2 for v in cluster) / len(cluster)
                variance = mu_squared - mu_tc_minus_t2**2
                sigma = math.sqrt(variance)
                averages.append(mu_tc_minus_t2)
                sigmas.append(sigma)
            mu_vals.append(averages)
            sigma_vals.append(sigmas)
    df_stats = pd.DataFrame(
        {
            'mu_vals' : mu_vals,
            'sigma_vals' : sigma_vals
        },
        index = cluster_dfs.index.values.tolist()
    )
    return(df_stats)

In [ ]:
calculate_averages(combined_df)

In [ ]:
def scenario_probability (df):
    '''
    Takes a dataframe...
    
    Inputs

    Outputs
    '''
    df['num_largest_clusters'] = df['cluster_coords'].apply(
        lambda x: max(len(i) for i in x)
    )
    df['scenario_probability'] = df.num_largest_clusters / df.qualifying_num
    df['overall_probability'] = df.scenario_probability * df.pos_conf
    return(df)

scenario_probability(combined_df)

**Calculate a pdf distribution for critical times**

- Critical time estimates using the full set of historical data can give a pdf of the distribution of the critical time
- Each point in time forecast (at a given t2), can give an estimate of the bubble critical time mu_tc, and the std deviation sigma_tc
- 1. create a plot of the k means clustering to vertify that the values of mu_tc and sigma_tc are sensible
  2. calculate the CDF of the forecast for the critical times
  3. Add this plot to the chart

**New function**

- Choose range of dates
- Function then takes all bubble forecasts within that period
- Combines them together to create an overall ensemble bubble forecast
- E.g if in a 10 day period, 6 days have positive bubble probabilities of 50%, then this may be a good indicator of potential bubble formation
- Rather than just using a single day's forecast, we give credence to having multiple forecasts on different run dates also overlapping.

In [ ]:
def view_bubble_clusters (df, index):
    '''
    A function to plot the qualifying positive bubble fits within the (t2- t1, tc-t2) space.
    K means clusters will appear as points near eachother if the predicted bubbles peak at the same time in the future.

    Inputs
    df (dataframe) :
        The dataframe which is the output of the 'get_clusters' function
    Outputs
    plt (matplotlib pyplot object)
        Returns a plot of qualifying bubble fits for a single time 't2' in the (t2-t1, tc-t2) space
        Plot also shows the coordinates of the centres of the clusters identified
    '''
    x = [item[0] for item in df.time_array[index]]
    y = [item[1] for item in df.time_array[index]]
    plt.scatter(
        x,
        y,
        name = 'All fits'
    )
    plt.scatter(
        x = [item[0] for item in df.cluster_coords[index]],
        y = [item[1] for item in df.cluster_coords[index]],
        name = 'Cluster Centres'
    )
    plt.legend()
    plt.show()
    return ()

def view_all_bubbles (df):
    indices = df.index.tolist()
    for i in indices:
        view_bubble_clusters(df, i)
        

In [ ]:
view_all_bubbles(cluster_df)

In [ ]:
tc_min_t2 = [tupl[1] for row in combined_df.time_array for tupl in row]
t2_min_t1 = [tupl[0] for row in combined_df.time_array for tupl in row]

plt.scatter(t2_min_t1,
            tc_min_t2)
plt.show()

combined_df.assign(qual_t2 = lambda combined_df.time_array + combined_df.time) #check

tc_qual = [pd.Timestamp.fromordinal(int(round(dicti['tc'],0))) for row in combined_df._fits for dicti in row if dicti['is_qualified'] == True]
t2_qual = [pd.Timestamp.fromordinal(int(round(dicti['t2'],0))) for row in combined_df._fits for dicti in row if dicti['is_qualified'] == True]

plt.scatter(
    t2_qual, tc_qual)
plt.axvline(x=pd.Timestamp.fromordinal(int(round(time[-1],0))), label='tc={}'.format(time[-1]), linestyle = '--', color = 'green')
plt.xticks(rotation=60)
plt.show()


**Create and save down a table**

- Save down a run of the bubble index at current date
- Append new rows of price data onto existing dataframe of price data
- 